# Introduction <a name="introduction"></a>

This notebook provides a comprehensive guide on utilizing the `watsonx.governance` Factsheets Python client to **create and manage models** and **AI use cases**. It illustrates the application of various **approaches** and **versioning** (Major, Minor, Patch) for effectively tracking models within an AI use case.

For detailed documentation on the `watsonx.governance` Factsheets Python client, please visit the [official documentation](https://s3.us.cloud-object-storage.appdomain.cloud/aifactsheets-client/index.html).

**Required Services:**
- `watsonx.governance`
- `watsonx.ai`

**Required Packages:**
- **IBM Facts Client Python SDK (>=1.0.47)**
- **IBM-watsonx-ai Python SDK**


  



In [1]:
!pip install -U ibm-aigov-facts-client --quiet
!pip install -U ibm-watsonx-ai --quiet
!pip install -U python-dotenv --quiet
!pip install -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 7.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.9.1
    Uninstalling matplotlib-3.9.1:
      Successfully uninstalled matplotlib-3.9.1


In [1]:
import warnings
import shutil
import time
import os
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
from ibm_watsonx_ai import APIClient
from ibm_aigov_facts_client import AIGovFactsClient,CloudPakforDataConfig
from IPython.display import display, Markdown


shutil.rmtree('./mlruns', ignore_errors=True)
load_dotenv()

/Users/joeleapen/.pyenv/versions/3.10.0/envs/TESTNB/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/joeleapen/.pyenv/versions/3.10.0/envs/TESTNB/lib/python3.10/site-packages/pydantic/_internal/_config.py:334: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


True

- This sample will use IBM Cloud by default. If you prefer to use Cloud pak for data, set `use_cp4d=True`
- Flag `run_cleanup_at_end` offers option to delete created assets at the end of the notebook.The notebook will show URL to UI for model and model use case at certain cells. By dafault we set it to `run_cleanup_at_end=False` so you can access UI and see the changes. If you decide to cleanup assets at the end, set `run_cleanup_at_end=True` and remember cells showing links to UI will `NOT` work in that case.

In [2]:
use_software=True
run_cleanup_at_end=True

- `Experiment` and `model names` are just for illustration purposes - they can be customized.
- Model container type can be `space` or `project`

In [3]:
experiment_name="IrisClassification"
MODEL_NAME="IrisScikitModel"
container_type="project"
container_id=os.getenv("CONTAINER_ID", "<if you can't use .env you can provide your value here>") # Project_id where the model will be stored

---
## Authentication Setup<a name="setup"></a>

### IBM Cloud  <a name="IBM-Cloud"></a>

Your Cloud API key can be generated by going to the Users section of the Cloud console. From that page, go to **Manage->Access(IAM)->API keys-> Create**. Give your key a name and click Create, then copy the created key and use as API_KEY.

NOTE: You can also get OpenScale API_KEY using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: instruction

How to get api key using console:

```
bx login --sso
bx iam api-key-create 'my_key'
```
- Get relevant space id from UI `(Deployments -> Spaces-> open space -> Manage -Space GUID)`

In [4]:
if not use_software:
    API_KEY=os.getenv("CLOUD_API_KEY", "<if you can't use .env you can provide your value here>")

[back to top](#introduction)
### Watsonx.Gov Platform <a name="Watsonx.Gov-Platform"></a>
- Service url is the Cloud pak for data platform host URL. For skytap environment, it would be the internal nginx URL.
- You can either use user `password` or platform `apikey` to authenticate

In [5]:
if use_software:
    
    creds=CloudPakforDataConfig(service_url=os.getenv("CPD_SERVICE_URL", "<if you can't use .env you can provide your value here>"),
        username=os.getenv("CPD_USERNAME", "<if you can't use .env you can provide your value here>"),
        password=os.getenv("CPD_PASSWORD", "<if you can't use .env you can provide your value here>"))


[back to top](#introduction)
## Client Initialization
- Container type would be either `space` or `project`. To use get/set environment utilities, model asset should be stored in Space.
- If running this notebook multiple times with same experiment name or anytime face error saying `Experiment with same name already exists`, use `set_as_current_experiment=True` when initiating client

[back to top](#introduction)



In [6]:
if use_software:
   facts_client = AIGovFactsClient(cloud_pak_for_data_configs=creds,experiment_name= experiment_name, container_type=container_type,container_id=container_id, set_as_current_experiment=True)
else: 
   facts_client = AIGovFactsClient(api_key=API_KEY, experiment_name= experiment_name, container_type=container_type,container_id=container_id, set_as_current_experiment=True)

2024/08/12 14:37:42 INFO : Experiment IrisClassification does not exist, creating new experiment
2024/08/12 14:37:42 INFO : Experiment successfully created with ID 102829089652578454 and name IrisClassification
2024/08/12 14:37:42 INFO : Autolog enabled Successfully


---

## Create and Train Model <a name="#createmodel"></a>

- This sample code demonstrates creating and training a model, specifically a classifier.
- Model development is achieved without writing any IBM or watsonx.governance-specific code.
- Key training facts are automatically captured in the background and can be saved to a factsheet later.


In [7]:
import pandas as pd
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score
import numpy as np

# Get testdata for iris.
iris=datasets.load_iris()

x=iris.data
y=iris.target

# Split training and test data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.5)

## Model Training

The following code will be used to train the model. During the training process, key facts and metrics will be automatically captured in the background by the `IBM watsonx.governance` factsheets client. These facts can later be saved and reviewed in a factsheet.

**Note:** Although there is no need to include any specific `IBM watsonx.governance` code for this process, you will still observe output


In [8]:
# Train model
classifier=tree.DecisionTreeClassifier()

#This is the main training method. No watxonx.governance code is directly needed but as you can see from the output the training details are logged
classifier.fit(x_train,y_train) 

# Predict model
predictions=classifier.predict(x_test)

# Check accuracy for the model
print(accuracy_score(y_test,predictions))

2024/08/12 14:37:51 INFO : logging results to factsheet for run_id d3540e8dd1764c7fb34a95b42c8512d8
2024/08/12 14:37:54 INFO : Successfully logged results to Factsheet service for run_id d3540e8dd1764c7fb34a95b42c8512d8 under asset_id: e4b88953-1b43-4519-9c6b-a79da3918134 and project_id : a50ca438-c31f-42e4-b72c-04bde15410f4
0.9333333333333333


---
## Store Model as a watsonx.ai / WML Asset in Project <a name="#savemodel"></a>

At present, the model exists solely as a Scikit-Learn object in memory. The subsequent step is to register it as a watsonx.ai / WML asset.

In a **Cloud environment**, please use the watsonx.ai / WML URL specific to your region:

- Dallas (US-South): `https://us-south.ml.cloud.ibm.com`

For IBM internal environments, the following watsonx.ai / WML URLs are applicable:

- ypqa: `https://us-south.ml.test.cloud.ibm.com`
- dev: `https://wml-fvt.ml.test.cloud.ibm.com`


In [11]:
from ibm_watsonx_ai import APIClient
from ibm_watsonx_ai import Credentials


if use_software:
       Credentials = {
                   "url": creds.url,
                   "username": creds.username,
                   "password" : creds.password,
                   "instance_id": "openshift",
                   "version" : "4.8"
                  }
else:
       Credentials = {
                     "url": "https://us-south.ml.cloud.ibm.com",
                     "apikey": API_KEY
              }

In [12]:
watsonx_ai_client = APIClient(Credentials)
watsonx_ai_client.version
watsonx_ai_client.set.default_project(container_id)

'SUCCESS'

### Define and Prepare Model Metadata for watsonx.ai / WML

The following code defines the software specification and model properties, and prepares the model metadata for registration with watsonx.ai / WML.


In [13]:
software_spec_uid = watsonx_ai_client.software_specifications.get_id_by_name("runtime-23.1-py3.10")
print("Software Specification ID: {}".format(software_spec_uid))

model_props = {
    watsonx_ai_client._models.ConfigurationMetaNames.NAME:"{}".format(MODEL_NAME),
    watsonx_ai_client._models.ConfigurationMetaNames.TYPE: "scikit-learn_1.1",
    watsonx_ai_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    watsonx_ai_client._models.ConfigurationMetaNames.LABEL_FIELD:"target",
}

facts_client.export_facts.prepare_model_meta(wml_client=watsonx_ai_client,meta_props=model_props)



Software Specification ID: 336b29df-e0e1-5e7d-b6a5-f6ab722625b2


{'name': 'IrisScikitModel',
 'type': 'scikit-learn_1.1',
 'software_spec': '336b29df-e0e1-5e7d-b6a5-f6ab722625b2',
 'label_column': 'target',
 'custom': {'experiment_id': '1e4f91d953054b278360f4f946c27566',
  'experiment_name': 'IrisClassification'}}


### Storing the Model and Retrieving the Model ID

The following code stores the model in Watsonx.ai / WML and retrieves its unique model asset ID.

**Note:** The IDs are unique only within their respective containers.
`

In [14]:
print("Storing model .....")

published_model_details = watsonx_ai_client.repository.store_model(model=classifier, meta_props=model_props, training_data=x_train, training_target=y_train)
model_id = watsonx_ai_client.repository.get_model_id(published_model_details)
print("Done")
print("Model ID: {}".format(model_id))

Storing model .....
Done
Model ID: c312255b-f2fb-46bb-bd15-7d13d5dabfda


---
## Retrieve Saved Model with Factsheet Client

The model, saved using watsonx.ai / WML methods, includes comprehensive training documentation in the factsheet. 
Although additional information can be manually added, such as diagrams, this is beyond the scope of this notebook.

To associate the model with an AI use case, retrieve it using the `assets.get_model()` method:

- Use `verbose=True` for detailed information.
- Retrieve the model by `model_id` with:
  - facts_client.assets.`get_model(model_id=<wml_model_id>)`
  
  - facts_client.assets.`get_model(model_id=<wml_model_id>, container_type=<space_or_project>, container_id=<space_or_project_id>)`


In [15]:
watsonx_ai_model=facts_client.assets.get_model(wml_stored_model_details=published_model_details)
watsonx_ai_model.get_info(verbose=True)

2024/08/12 14:38:57 INFO : Current model information: {'asset_id': 'c312255b-f2fb-46bb-bd15-7d13d5dabfda', 'container_type': 'project', 'container_id': 'a50ca438-c31f-42e4-b72c-04bde15410f4', 'facts_type': 'modelfacts_user'}


{'name': 'IrisScikitModel',
 'asset_type': 'wml_model',
 'url': 'https://cpd-aigov.apps.fs-hotfix-test1.cp.fyre.ibm.com/ml/models/c312255b-f2fb-46bb-bd15-7d13d5dabfda?projectid=a50ca438-c31f-42e4-b72c-04bde15410f4&context=cpdaas',
 'asset_id': 'c312255b-f2fb-46bb-bd15-7d13d5dabfda',
 'container_type': 'project',
 'container_id': 'a50ca438-c31f-42e4-b72c-04bde15410f4',
 'facts_type': 'modelfacts_user'}

In [16]:
model_ui_url = watsonx_ai_model.get_info(verbose=True)["url"]
display(Markdown("[Click here to see the created model asset and it's factsheet in the UI](" + model_ui_url + ")"))

[Click here to see the created model asset and it's factsheet in the UI](https://cpd-aigov.apps.fs-hotfix-test1.cp.fyre.ibm.com/ml/models/c312255b-f2fb-46bb-bd15-7d13d5dabfda?projectid=a50ca438-c31f-42e4-b72c-04bde15410f4&context=cpdaas)



---
## Creation of New AI Use Case <a href="#add_mut"></a>

An **AI Use Case** tracks model asset lifecycles across environments like development, pre-production, and production. 

**Note:** The term "AI Use Case" has replaced "Model Use Case" to reflect a broader range of AI assets. While some APIs may still use the old terminology, it will be phased out.

- If `ai_usecase_id` is not provided, the default inventory_id is used (requires `EDITOR` access).

- Retrieve the AI Use Case ID from the URL in inventory or by using `get_ai_usecase()`.

- For Cloud Pak for Data, ensure OpenPages integration is disabled (create inventory permission needed).


In [17]:
ai_usecase_inventory_id = os.getenv("INVENTORY_ID", "<if you can't use .env you can provide your inventory id here>")
ai_usecase_name="Automatic Iris classification - demonstration use case" 
ai_usecase_desc="AI usecase for iris classification"

##### ⚠️ Attention: Use of inventories vs. catalogs as input to the `catalog_id` parameter

- The `catalog_id` parameter can specify either an inventory ID or a catalog ID. Inventories which technically are a sub-type of catalogs optimized for watsonx.governance, are recommended.


- Catalogs are still supported but will be deprecated over time. As a best practice, use inventories for storing use cases and external models.


In [22]:
ai_usecase = facts_client.assets.create_ai_usecase(catalog_id="b1e3758b-ae8e-4708-8ade-d954324e879f",name=ai_usecase_name,description=ai_usecase_desc)
ai_usecase.get_info(True)

2024/08/12 14:41:33 INFO : AI usecase created successfully


{'name': 'Automatic Iris classification - demonstration use case',
 'description': 'AI usecase for iris classification',
 'asset_type': 'model_entry',
 'url': 'https://cpd-aigov.apps.fs-hotfix-test1.cp.fyre.ibm.com/data/catalogs/b1e3758b-ae8e-4708-8ade-d954324e879f/asset/2554e2d2-81ac-4c53-ae7b-851b1fc651d5?context=cpdaas',
 'model_usecase_id': '2554e2d2-81ac-4c53-ae7b-851b1fc651d5',
 'container_type': 'catalog',
 'catalog_id': 'b1e3758b-ae8e-4708-8ade-d954324e879f',
 'facts_type': 'model_entry_user'}

#### Methods Available for AI Use Cases

Explore the following methods for managing AI Use Cases:

In [23]:
print("AI usecase name is : {}".format(ai_usecase.get_name()))
print("AI usecase ID is : {}".format(ai_usecase.get_id()))
print("AI usecase catalog is : {}".format(ai_usecase.get_container_id()))
print("AI usecase container type is : {}".format(ai_usecase.get_container_type()))
print("AI usecase description is : {}".format(ai_usecase.get_description()))

AI usecase name is : Automatic Iris classification - demonstration use case
AI usecase ID is : 2554e2d2-81ac-4c53-ae7b-851b1fc651d5
AI usecase catalog is : b1e3758b-ae8e-4708-8ade-d954324e879f
AI usecase container type is : catalog
AI usecase description is : AI usecase for iris classification


---
## Create an Approach <a href="#createapproach"></a>

- Track multiple models and prompts under a single use case by grouping them into different approaches.
- Create multiple approaches for various classification algorithms to facilitate comparison and integration.
- Use approaches to manage different models that need to be combined for a specific use case.


#### Create an Approach for Decision Tree classification
- Define a new approach specifically for Decision Tree classification within the existing use case.
- This approach will enable tracking and management of the Decision Tree model alongside other classification algorithms.

In [24]:
decisiontree_approach = ai_usecase.create_approach(name="Decision Tree classification",description="Use a descision tree approach to classify iris data",icon="Sprout",color="Teal")
decisiontree_approach.get_info()

2024/08/12 14:41:50 INFO : Approach created successfully


{'approach_id': '7bbd0688-55f6-4315-9123-1150b71314ce',
 'approach_name': 'Decision Tree classification',
 'approach_desc': 'Use a descision tree approach to classify iris data',
 'model_asset_id': '2554e2d2-81ac-4c53-ae7b-851b1fc651d5',
 'model_container_type': 'catalog',
 'model_container_id': 'b1e3758b-ae8e-4708-8ade-d954324e879f'}

#### Create an Approach for Random Forest Classification

- Define a new approach specifically for Random Forest classification within the existing use case.
- This approach will enable tracking and management of the Random Forest model alongside other classification algorithms.


In [25]:
randomforest_approach = ai_usecase.create_approach(name="Random Forest classification",description="Use a Random Forest approach to classify iris data",icon="Tree",color="Green")
randomforest_approach.get_info()

2024/08/12 14:41:52 INFO : Approach created successfully


{'approach_id': 'bb93d13d-b450-4da6-bfd7-49a002c81b71',
 'approach_name': 'Random Forest classification',
 'approach_desc': 'Use a Random Forest approach to classify iris data',
 'model_asset_id': '2554e2d2-81ac-4c53-ae7b-851b1fc651d5',
 'model_container_type': 'catalog',
 'model_container_id': 'b1e3758b-ae8e-4708-8ade-d954324e879f'}

### Retrieve All Approaches

- Fetch a list of all approaches associated with the current use case.
- This allows you to review and manage the various models and methods tracked under the use case.
#thampp: after creating two approaches in this notebook the users will have three because the default approach is still there. We should teach our users how they can get the two approaches they want (or any number). I don't think we can delete the default. So maybe we will have to change the example such that it renames the default


In [26]:
approaches = ai_usecase.get_approaches()
for approach_detail in approaches:
    print(approach_detail.get_info())

2024/08/12 14:41:56 INFO : Approaches retrieved successfully
{'approach_id': 'bb93d13d-b450-4da6-bfd7-49a002c81b71', 'approach_name': 'Random Forest classification', 'approach_desc': 'Use a Random Forest approach to classify iris data', 'model_asset_id': '2554e2d2-81ac-4c53-ae7b-851b1fc651d5', 'model_container_type': 'catalog', 'model_container_id': 'b1e3758b-ae8e-4708-8ade-d954324e879f'}
{'approach_id': '7bbd0688-55f6-4315-9123-1150b71314ce', 'approach_name': 'Decision Tree classification', 'approach_desc': 'Use a descision tree approach to classify iris data', 'model_asset_id': '2554e2d2-81ac-4c53-ae7b-851b1fc651d5', 'model_container_type': 'catalog', 'model_container_id': 'b1e3758b-ae8e-4708-8ade-d954324e879f'}
{'approach_id': '00000000-0000-0000-0000-000000000000', 'approach_name': 'Default approach', 'approach_desc': 'A default approach for tracking your AI assets.', 'model_asset_id': '2554e2d2-81ac-4c53-ae7b-851b1fc651d5', 'model_container_type': 'catalog', 'model_container_id': 

### Retrieve Single Approach

- Retrieve details of a specific approach within the use case.
- This allows you to access information about a particular model or method tracked under the use case.


In [28]:
decisiontree_approach = ai_usecase.get_approach(approach_id=approaches[1].get_id())
decisiontree_approach.get_info()

2024/08/12 14:43:34 INFO : Approach retrieved successfully


{'approach_id': '00000000-0000-0000-0000-000000000000',
 'approach_name': 'Default approach',
 'approach_desc': 'A default approach for tracking your AI assets.',
 'model_asset_id': '2554e2d2-81ac-4c53-ae7b-851b1fc651d5',
 'model_container_type': 'catalog',
 'model_container_id': 'b1e3758b-ae8e-4708-8ade-d954324e879f'}

> Here, you can see that there are three approaches: two are user-created, and one is the default approach. The default approach cannot be deleted.
 **Note:** We can rename the default approach to **"Custom Approach"** as per user requirements and use it accordingly. The original default approach itself cannot be deleted.


In [ ]:
default_approach = ai_usecase.get_approach(approach_id=approaches[2].get_id())
default_approach.get_info()

In [ ]:
default_approach.set_name(name="Model Regression classicifation")
default_approach.set_description(description="Use a Regression  tree approach to classify iris data")

default_approach.get_info()

### Delete an Approach

- Remove a specific approach from the use case.
- This action deletes the associated model or method, helping to manage and streamline your tracking process.


In [30]:
ai_usecase.remove_approach(approach=decisiontree_approach)

2024/08/12 14:45:17 INFO : Can't delete default approach 


In [40]:
ai_usecase.remove_approach(approach=approaches[0])

2024/08/05 15:05:38 INFO : Approach removed successfully


### Other Methods available for approach
Explore the following methods for managing approach:

In [ ]:
approaches = ai_usecase.get_approaches()
decisiontree_approach = ai_usecase.get_approach(approach_id=approaches[0].get_id())
print(decisiontree_approach)

## Update approach name and description
decisiontree_approach.set_name(name="Decision Tree classification")
decisiontree_approach.set_description(description="Use a descision tree approach to classify iris data")

## Get approach versions
decisiontree_approach.get_versions()

## Get approach info
decisiontree_approach.get_info()

In [42]:
print("Approach name is : {}".format(decisiontree_approach.get_name()))
print("Approach ID is : {}".format(decisiontree_approach.get_id()))
print("Approach description is : {}".format(decisiontree_approach.get_description()))
print("Approach usecase is : {}".format(decisiontree_approach.get_model_useacse_id()))
print("Approach usecase container type is : {}".format(decisiontree_approach.get_model_usecase_container_type()))
print("Approach usecase container ID is : {}".format(decisiontree_approach.get_model_usecase_container_id()))

Approach name is : Decision Tree classification
Approach ID is : afb4cd3a-79d7-493a-b6e0-b5d79ed70763
Approach description is : Use a descision tree approach to classify iris data
Approach usecase is : ae383786-2bf7-408b-9115-159c557bf561
Approach usecase container type is : catalog
Approach usecase container ID is : 0d136f5c-3756-46a8-ab50-979fd585b26e


---
## Track a Model Under an AI Use Case <a name="add_mu"></a>

- **AI Use Cases** are designed to monitor the lifecycle of model assets across various stages, including development, pre-production, and production.
- To effectively integrate a model into an AI use case, three critical elements must be addressed: the **model**, the **AI use case**, and the **approach**.
- Link an existing AI use case by using the following method: `model.track(usecase=<ai_usecase>, approach=<approach1>, version_number="<Version Number>")`.

- **Version Numbers** are categorized as follows:
    - **Major Version:** Indicates significant changes, represented as `1.0.0`.
    - **Minor Version:** Reflects incremental improvements, represented as `0.1.0`.
    - **Patch Version:** Denotes minor fixes or updates, represented as `0.0.1`.
    - **Custom Version:** Allows for tailored versioning according to specific user needs.

- Ensure that the `ai_usecase`, `approach`, and `version_number` parameters are provided as mandatory.



In [ ]:
watsonx_ai_model.track(usecase=ai_usecase,approach=decisiontree_approach,version_number="major",version_comment="major update to previous version")

#### Retrieve Tracked Models for Use Case

- Fetch all models that are currently tracked under a specified AI use case.
- This provides an overview of all models associated with the use case, facilitating management and analysis.


In [ ]:
ai_usecase.get_tracked_models()

## Untrack a Model

Remove a model from an AI use case when it is no longer relevant or needs to be managed separately.


In [ ]:
watsonx_ai_model.untrack()

---
## Cleanup<a href="#clean"></a>

In [ ]:
if run_cleanup_at_end:
    facts_client.assets.remove_asset(asset_id=ai_usecase.get_info()["model_usecase_id"],container_type=ai_usecase.get_info()["container_type"],container_id=ai_usecase.get_info()["catalog_id"])
    facts_client.assets.remove_asset(asset_id=watsonx_ai_model.get_info()["asset_id"],container_type=watsonx_ai_model.get_info()["container_type"],container_id=watsonx_ai_model.get_info()["container_id"])
else:
    model_ui_url = watsonx_ai_model.get_info(verbose=True)["url"]
    display(Markdown("[Click here to see the created wml model details in the UI](" + model_ui_url + ")"))
    ai_usecase_ui_url = ai_usecase.get_info(verbose=True)["url"]
    display(Markdown("[Click here to see the created AI use case in the UI](" + ai_usecase_ui_url + ")"))

**Created by:**  


IBM watsonx.governance - AI Factsheet Python SDK Team

---

**Copyright © 2020-2024 IBM**  
Released under the MIT License.
